In [6]:
from websocket import create_connection
import json, time
import pandas as pd
from openpyxl import load_workbook
from openpyxl.utils.dataframe import dataframe_to_rows
from itertools import islice
import numpy as np
import csv

In [ ]:
"""trades = list()
sec = 0

for i in range(3):
    try:
        ws = create_connection('wss://api-pub.bitfinex.com/ws/2')
    except Exception as error:
        print('Caught this error: ' + repr(error))
        time.sleep(3)
    else:
        break

ws.send(json.dumps({"event": "subscribe", "channel": "book", "prec": "R0", "symbol": "tBTCUSD", 'len':'100'}))


count = 0
while True:
    try:
        if count == 4:
            break
        result = ws.recv()
        result = json.loads(result)
        trades.append(result)
        sec = time.time()
        count += 1
    except Exception as error:
        print('Caught this error: ' + repr(error))
        time.sleep(3)

ws.close()

In [ ]:
"""trades = trades[2][1:]
times = np.repeat(time.time(), 200)

In [ ]:
"""df_trades = pd.DataFrame(trades[0], columns = ['order', 'price', 'volume'])
df_trades = pd.concat([df_trades, pd.DataFrame(times)], axis = 1)
df_trades.columns = ['order', 'price', 'volume', 'time', ]
df_trades.head()

In [ ]:
"""values = df_trades.values.tolist()
with open('C:/Users/Dell/Desktop/Diploma paper/new_trades.csv', mode='a+', newline='') as file:
    file_writer = csv.writer(file, delimiter=',')
    for rows in values:
        file_writer.writerow(rows)

In [ ]:
'''asks = df_trades[(df_trades['volume'] < 0) & (df_trades['price'] != 0)]
bids = df_trades[(df_trades['volume'] > 0) & (df_trades['price'] != 0)]
cancelled =df_trades[df_trades['price'] == 0]

In [ ]:
'''wb = load_workbook(filename = 'C:/Users/Dell/Desktop/Diploma paper/trades.xlsx')
ws_a = wb['Asks']
ws_b = wb['Bids']
ws_can = wb['Cancelled']

In [ ]:
'''for r in dataframe_to_rows(bids, index=False, header=True): #change header to True for the 1st time
    ws_b.append(r)
for r in dataframe_to_rows(asks, index=False, header=True):
    ws_a.append(r)
for r in dataframe_to_rows(cancelled, index=False, header=True):
    ws_can.append(r)
wb.save('trades.xlsx')

In [ ]:
'''def create_df_from_excel(ws):
    data = ws.values
    cols = next(data)[1:]
    data = list(data)
    idx = [r[0] for r in data]
    data = (islice(r, 1, None) for r in data)
    df = pd.DataFrame(data, index=idx, columns=cols)
    df.index = range(df.shape[0])
    df = df.join(pd.DataFrame(idx, columns = ['order']))
    return df

In [ ]:
'''if cancelled.shape[0] != 0:
    wb_renewed = load_workbook(filename = 'C:/Users/Dell/Desktop/Diploma paper/trades.xlsx')
    if cancelled[cancelled['volume'] < 0].shape[0]>0:
        ws_renewed_a = wb_renewed['Asks']
        asks_renewed = pd.read_from_excel(ws_renewed_a)
    if cancelled[cancelled['volume'] > 0].shape[0]>0:
        ws_renewed_b = wb_renewed['Bids']
        bids_renewed = create_df_from_excel(ws_renewed_b)
    for r_canc in range(cancelled.shape[0]):
        if cancelled.iloc[r_canc, 2] > 0:
            ind_c = bids_renewed[bids_renewed['order'] == cancelled.iloc[r_canc, 0]].index[0] + 2
            ws_renewed_b.cell(row=ind_c, column=4, value=cancelled.iloc[r_canc, 0])
            ws_renewed_b['E' + str(ind_c)] = cancelled.iloc[r_canc, 1]
            ws_renewed_b['F' + str(ind_c)] = cancelled.iloc[r_canc, 2]
            print(ws_renewed_b['F13'].value)
            
        wb.save('trades.xlsx')

In [7]:
def GetOrderBook():
    orderbook = list()
    for i in range(3):
            try:
                ws = create_connection('wss://api-pub.bitfinex.com/ws/2')
            except Exception as error:
                print('Caught this error: ' + repr(error))
                time.sleep(3)
            else:
                break

    ws.send(json.dumps({'event': 'subscribe','channel': 'book', 'symbol': 'tBTCUSD', 'prec':'P1', 'len':'25'}))


    count = 0
    while True:
        try:
            if count == 4:
                break
            result = ws.recv()
            result = json.loads(result)
            orderbook.append(result)
            count += 1
        except Exception as error:
            print('Caught this error: ' + repr(error))
            time.sleep(3)

    ws.close()
    return orderbook[2:][0][1:]

In [8]:
def merge_columns_orders(t, df):
    l = [t]
    for i in range(df.shape[0]):
        l.append(df.iloc[i, 0])
        l.append(df.iloc[i, 1])
        l.append(df.iloc[i, 2])
    return l

def writeOrderBook(t, orderbook):
    orderbook = pd.DataFrame(orderbook[0], columns = ['price', 'num_orders','volume'])
    #orderbook = pd.concat([orderbook, pd.DataFrame(t)], axis = 1)
    #orderbook.columns = ['price', 'num_orders','volume', 'time', ]
    
    oasks = orderbook[(orderbook['volume'] < 0)]
    obids = orderbook[(orderbook['volume'] > 0)]
    
    
    ovalues_asks = merge_columns_orders(t, oasks)
        
    with open('C:/Users/Dell/Desktop/Diploma paper/OrderBookAsks.csv', mode='a+', newline='') as ofile_asks:
        ofile_writer_asks = csv.writer(ofile_asks, delimiter=',')
        #for orows_a in ovalues_asks:
        ofile_writer_asks.writerow(ovalues_asks)
            
    ovalues_bids = merge_columns_orders(t, obids)
        
    with open('C:/Users/Dell/Desktop/Diploma paper/OrderBookBids.csv', mode='a+', newline='') as ofile_bids:
        ofile_writer_bids = csv.writer(ofile_bids, delimiter=',')
        #for orows_b in ovalues_bids:
        ofile_writer_bids.writerow(ovalues_bids)

In [9]:
while True:
    trades = list()
    
    for i in range(3):
        try:
            ws = create_connection('wss://api-pub.bitfinex.com/ws/2')
        except Exception as error:
            print('Caught this error: ' + repr(error))
            time.sleep(3)
        else:
            break

    ws.send(json.dumps({"event": "subscribe", "channel": "book", "prec": "R0", "symbol": "tBTCUSD", 'len':'100'}))


    count = 0
    while True:
        try:
            if count == 4:
                break
            result = ws.recv()
            result = json.loads(result)
            trades.append(result)
            count += 1
        except Exception as error:
            print('Caught this error: ' + repr(error))
            time.sleep(3)

    ws.close()
    
    orderbook = GetOrderBook()
    
    trades = trades[2][1:]
    times = time.time()
    df_trades = pd.DataFrame(trades[0], columns = ['order', 'price', 'volume'])
    #df_trades = pd.concat([df_trades, pd.DataFrame(times)], axis = 1)
    #df_trades.columns = ['order', 'price', 'volume', 'time', ]
    
    asks = df_trades[(df_trades['volume'] < 0) & (df_trades['price'] != 0)]
    bids = df_trades[(df_trades['volume'] > 0) & (df_trades['price'] != 0)]
    cancelled =df_trades[df_trades['price'] == 0]
    
    
    values_asks = merge_columns_orders(times, asks)#asks.values.tolist()
    
    with open('C:/Users/Dell/Desktop/Diploma paper/Asks.csv', mode='a+', newline='') as file_asks:
        file_writer_asks = csv.writer(file_asks, delimiter=',')
        #for rows_a in values_asks:
        file_writer_asks.writerow(values_asks)
            
    values_bids = merge_columns_orders(times, bids)#bids.values.tolist()
    with open('C:/Users/Dell/Desktop/Diploma paper/Bids.csv', mode='a+', newline='') as file_bids:
        file_writer_bids = csv.writer(file_bids, delimiter=',')
        #for rows_b in values_bids:
        file_writer_bids.writerow(values_bids)
    
    if cancelled.shape[0] > 0:
        values_can = merge_columns_orders(times, cancelled)#cancelled.values.tolist()
        with open('C:/Users/Dell/Desktop/Diploma paper/Cancelled.csv', mode='a+', newline='') as file_can:
            file_writer_can = csv.writer(file_can, delimiter=',')
            #for rows_c in values_can:
            file_writer_can.writerow(values_can)
            
    writeOrderBook(times, orderbook)

    #time.sleep(2)

ght this error: WebSocketConnectionClosedException('socket is already closed.')
Caught this error: WebSocketConnectionClosedException('socket is already closed.')
Caught this error: WebSocketConnectionClosedException('socket is already closed.')
Caught this error: WebSocketConnectionClosedException('socket is already closed.')
Caught this error: WebSocketConnectionClosedException('socket is already closed.')
Caught this error: WebSocketConnectionClosedException('socket is already closed.')
Caught this error: WebSocketConnectionClosedException('socket is already closed.')
Caught this error: WebSocketConnectionClosedException('socket is already closed.')
Caught this error: WebSocketConnectionClosedException('socket is already closed.')
Caught this error: WebSocketConnectionClosedException('socket is already closed.')
Caught this error: WebSocketConnectionClosedException('socket is already closed.')
Caught this error: WebSocketConnectionClosedException('socket is already closed.')
Caught 

KeyboardInterrupt: 

In [38]:
cancelled.shape

(0, 3)